필요한 모듈과 라이브러리들을 가져 옴

In [1]:
#import the libraries For data analysis 
import sys
import pandas as pd
import matplotlib
import numpy as np
import scipy as sp
import IPython
import sklearn

In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

matplotlib에서 한글이 깨지는 상황을 막기위해 코드 삽입

In [3]:
%matplotlib inline

import platform
path="c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc

if platform.system()=='Darwin':
    rc('font', family='AppleGothic')
elif platform.system() =='Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family = font_name)
else:
    print('Unknown system')

이미 가지고 있는 교통사고, 인구, 범죄, 세금 데이터들을 가공해 만든 임의의 통합데이터를 사용함

In [6]:
#Load the convert data
data_2015 = pd.read_csv('./data/통합/2015통합.csv', engine='python')
data_2016=  pd.read_csv('./data/통합/2016통합.csv', engine='python')


In [12]:
data_2015.head()

,crimeRate,city_idx,total,out_per,total_m,total_w,out_m,out_w,suicide_rate,tax,death,injured,cctv,city_id,city_name,district_name,year,per_tax
city,,,,,,,,,,,,,,,,,,
서울특별시 서울,0.007215,1,9904312,0.0340,4859535,5044777,165218,171898,19.9,9169412842,376,591,44457,1,서울특별시,서울,2015,925.800080
인천광역시 인천,0.006780,2,2890451,0.0235,1455017,1435434,40224,27626,24.1,1764120424,136,255,8680,2,인천광역시,인천,2015,610.327047
제주특별자치도 제주,0.009765,3,605619,0.0250,304789,300830,8914,6232,22.3,509905893,93,145,2249,3,제주특별자치도,제주,2015,841.958216
세종특별자치시 세종,0.002440,4,204088,0.0219,103210,100878,2755,1716,19.7,328603432,19,24,1848,4,세종특별자치시,세종,2015,1610.106581
대전광역시 대전,0.006199,5,1538394,0.0124,772243,766151,8933,10147,24.5,585202673,97,143,6356,5,대전광역시,대전,2015,380.398437


두개의 데이터를 합쳐 하나의 total data로 한 후 작업을 실시

그러기 위해서는 데이터 구분을 위한 column 추가

In [7]:
data_2015['year'] = 2015
data_2016['year'] = 2016
data_2015['per_tax'] = data_2015['tax'] / data_2015['total']
data_2016['per_tax'] = data_2016['tax'] / data_2016['total']
data_2015.head()

,crimeRate,city_idx,total,out_per,total_m,total_w,out_m,out_w,suicide_rate,tax,death,injured,cctv,city_id,city_name,district_name,year,per_tax
0,0.007215,1,9904312,0.0340,4859535,5044777,165218,171898,19.9,9169412842,376,591,44457,1,서울특별시,서울,2015,925.800080
1,0.006780,2,2890451,0.0235,1455017,1435434,40224,27626,24.1,1764120424,136,255,8680,2,인천광역시,인천,2015,610.327047
2,0.009765,3,605619,0.0250,304789,300830,8914,6232,22.3,509905893,93,145,2249,3,제주특별자치도,제주,2015,841.958216
3,0.002440,4,204088,0.0219,103210,100878,2755,1716,19.7,328603432,19,24,1848,4,세종특별자치시,세종,2015,1610.106581
4,0.006199,5,1538394,0.0124,772243,766151,8933,10147,24.5,585202673,97,143,6356,5,대전광역시,대전,2015,380.398437


구분하기 편하게 district_name과 city_name을 합쳐 하나의 city 속성으로 하고

그 속성을 index로 변환

In [8]:
data_2015['city'] = data_2015['city_name'] +' ' +data_2015['district_name']
data_2015.set_index('city', inplace=True)
data_2016['city'] = data_2016['city_name'] +' ' +data_2016['district_name']
data_2016.set_index('city', inplace=True)

In [10]:
total_data = pd.concat([data_2015, data_2016])

C:\Users\Pandahun\Anaconda3\envs\tensorflow36\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [11]:
total_data.head()

,cctv,city_id,city_idx,city_name,crimeRate,death,district_name,injured,out_m,out_per,out_w,per_tax,suicide_rate,tax,total,total_m,total_old,total_w,year
city,,,,,,,,,,,,,,,,,,,
서울특별시 서울,44457,1,1,서울특별시,0.007215,376,서울,591,165218,0.0340,171898,925.800080,19.9,9169412842,9904312,4859535,NaN,5044777,2015
인천광역시 인천,8680,2,2,인천광역시,0.006780,136,인천,255,40224,0.0235,27626,610.327047,24.1,1764120424,2890451,1455017,NaN,1435434,2015
제주특별자치도 제주,2249,3,3,제주특별자치도,0.009765,93,제주,145,8914,0.0250,6232,841.958216,22.3,509905893,605619,304789,NaN,300830,2015
세종특별자치시 세종,1848,4,4,세종특별자치시,0.002440,19,세종,24,2755,0.0219,1716,1610.106581,19.7,328603432,204088,103210,NaN,100878,2015
대전광역시 대전,6356,5,5,대전광역시,0.006199,97,대전,143,8933,0.0124,10147,380.398437,24.5,585202673,1538394,772243,NaN,766151,2015


여기서 필요한 정보를 제외하고는 삭제를 진행

전체 남성, 전체 여성 ,내국인 남성, 외국인 여성, 외국인 남성 속성을 삭제

In [ ]:
#delete the non_valueable
del data_2015['total_m']
del data_2015['total_w']
del data_2015['out_m']
del data_2015['out_w']
del data_2015['city_idx']
del data_2016['total_m']
del data_2016['total_w']
del data_2016['out_m']
del data_2016['out_w']
del data_2016['city_idx']

### Correlation Analysis

상관분석은 numpy 모듈의 corrcoef를 사용함

상관분석에서

절댓값 0.1 이하는 무시하고

절댓값 0.3 이하는 약한 상관관계를

절댓값 0.7 이하는 강한 상관관계를 나타낸다

세금을 1인당 세금 비율로 계산

##### 우선은 범죄율과 다른 데이터들의상관관계를 분석

In [ ]:
print('2015년 상관관계')
print('범죄율 / 인구수: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['total']))
print('범죄율 / 외국인 비율: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['out_per']))
print('범죄율 / 자살율: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['suicide_rate']))
print('범죄율 / 소득수준: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['per_tax']))
print('범죄율 / 교통사고사망자: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['death']))
print('범죄율 / 교통사고부상자: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['injured']))
print('범죄율 / cctv: ')
print(np.corrcoef(data_2015['crimeRate'], data_2015['cctv']))


그 다음은 인구수와 상관관계를 보려고 한다

In [ ]:
print('인구수 / 외국인비율: ')
print(np.corrcoef(data_2015['total'], data_2015['out_per']))
print('인구수 / 자살율: ')
print(np.corrcoef(data_2015['total'], data_2015['suicide_rate']))
print('인구수 / 소득수준: ')
print(np.corrcoef(data_2015['total'], data_2015['per_tax']))
print('인구수 / 교통사고사망자: ')
print(np.corrcoef(data_2015['total'], data_2015['death']))
print('인구수 / 교통사고부상자: ')
print(np.corrcoef(data_2015['total'], data_2015['injured']))


In [ ]:
print('외국인비율 / 자살율: ')
print(np.corrcoef(data_2015['out_per'], data_2015['suicide_rate']))
print('외국인비율 / 소득수준: ')
print(np.corrcoef(data_2015['out_per'], data_2015['per_tax']))
print('외국인비율 / 교통사고사망자: ')
print(np.corrcoef(data_2015['out_per'], data_2015['death']))
print('외국인비율 / 교통사고부상자: ')
print(np.corrcoef(data_2015['out_per'], data_2015['injured']))
print('외국인비율 / cctv: ')
print(np.corrcoef(data_2015['out_per'], data_2015['cctv']))

In [ ]:
print('자살율 / 소득수준: ')
print(np.corrcoef(data_2015['suicide_rate'], data_2015['per_tax']))
print('자살율 / 교통사고사망자: ')
print(np.corrcoef(data_2015['suicide_rate'], data_2015['death']))
print('자살율 / 교통사고부상자: ')
print(np.corrcoef(data_2015['suicide_rate'], data_2015['injured']))
print('자살율 / cctv: ')
print(np.corrcoef(data_2015['suicide_rate'], data_2015['cctv']))

In [ ]:
print('소득수준 / 교통사고사망자: ')
print(np.corrcoef(data_2015['per_tax'], data_2015['death']))
print('소득수준 / 교통사고부상자: ')
print(np.corrcoef(data_2015['per_tax'], data_2015['injured']))
print('소득수준 / cctv: ')
print(np.corrcoef(data_2015['per_tax'], data_2015['cctv']))

In [ ]:
print('교통사고사망자 / 교통사고부상자: ')
print(np.corrcoef(data_2015['death'], data_2015['injured']))
print('교통사고사망자 / cctv: ')
print(np.corrcoef(data_2015['death'], data_2015['cctv']))

In [ ]:
print('교통사고부상자 / cctv: ')
print(np.corrcoef(data_2015['injured'], data_2015['cctv']))

##### 2016년도

In [ ]:
print('2016년 상관관계')
print('범죄율 / 인구수: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['total']))
print('범죄율 / 외국인 비율: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['out_per']))
print('범죄율 / 자살율: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['suicide_rate']))
print('범죄율 / 소득수준: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['per_tax']))
print('범죄율 / 교통사고사망자: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['death']))
print('범죄율 / 교통사고부상자: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['injured']))
print('범죄율 / cctv: ')
print(np.corrcoef(data_2016['crimeRate'], data_2016['cctv']))


In [ ]:
print('인구수 / 외국인비율: ')
print(np.corrcoef(data_2016['total'], data_2016['out_per']))
print('인구수 / 자살율: ')
print(np.corrcoef(data_2016['total'], data_2016['suicide_rate']))
print('인구수 / 소득수준: ')
print(np.corrcoef(data_2016['total'], data_2016['per_tax']))
print('인구수 / 교통사고사망자: ')
print(np.corrcoef(data_2016['total'], data_2016['death']))
print('인구수 / 교통사고부상자: ')
print(np.corrcoef(data_2016['total'], data_2016['injured']))


In [ ]:
print('외국인비율 / 자살율: ')
print(np.corrcoef(data_2016['out_per'], data_2016['suicide_rate']))
print('외국인비율 / 소득수준: ')
print(np.corrcoef(data_2016['out_per'], data_2016['per_tax']))
print('외국인비율 / 교통사고사망자: ')
print(np.corrcoef(data_2016['out_per'], data_2016['death']))
print('외국인비율 / 교통사고부상자: ')
print(np.corrcoef(data_2016['out_per'], data_2016['injured']))
print('외국인비율 / cctv: ')
print(np.corrcoef(data_2016['out_per'], data_2016['cctv']))

In [ ]:
print('자살율 / 소득수준: ')
print(np.corrcoef(data_2016['suicide_rate'], data_2016['per_tax']))
print('자살율 / 교통사고사망자: ')
print(np.corrcoef(data_2016['suicide_rate'], data_2016['death']))
print('자살율 / 교통사고부상자: ')
print(np.corrcoef(data_2016['suicide_rate'], data_2016['injured']))
print('자살율 / cctv: ')
print(np.corrcoef(data_2016['suicide_rate'], data_2016['cctv']))

In [ ]:
print('소득수준 / 교통사고사망자: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['death']))
print('소득수준 / 교통사고부상자: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['injured']))
print('소득수준 / cctv: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['cctv']))

In [ ]:
print('교통사고사망자 / 교통사고부상자: ')
print(np.corrcoef(data_2016['death'], data_2016['injured']))
print('교통사고사망자 / cctv: ')
print(np.corrcoef(data_2016['death'], data_2016['cctv']))

In [ ]:
print('교통사고부상자 / cctv: ')
print(np.corrcoef(data_2016['injured'], data_2016['cctv']))

In [ ]:
print('전체 상관관계')
print('범죄율 / 인구수: ')
print(np.corrcoef(total_data['crimeRate'], total_data['total']))
print('범죄율 / 외국인 비율: ')
print(np.corrcoef(total_data['crimeRate'], total_data['out_per']))
print('범죄율 / 자살율: ')
print(np.corrcoef(total_data['crimeRate'], total_data['suicide_rate']))
print('범죄율 / 소득수준: ')
print(np.corrcoef(total_data['crimeRate'], total_data['per_tax']))
print('범죄율 / 교통사고사망자: ')
print(np.corrcoef(total_data['crimeRate'], total_data['death']))
print('범죄율 / 교통사고부상자: ')
print(np.corrcoef(total_data['crimeRate'], total_data['injured']))
print('범죄율 / cctv: ')
print(np.corrcoef(total_data['crimeRate'], total_data['cctv']))


In [ ]:
print('인구수 / 외국인비율: ')
print(np.corrcoef(total_data['total'], total_data['out_per']))
print('인구수 / 자살율: ')
print(np.corrcoef(total_data['total'], total_data['suicide_rate']))
print('인구수 / 소득수준: ')
print(np.corrcoef(total_data['total'], total_data['per_tax']))
print('인구수 / 교통사고사망자: ')
print(np.corrcoef(total_data['total'], total_data['death']))
print('인구수 / 교통사고부상자: ')
print(np.corrcoef(total_data['total'], total_data['injured']))


In [ ]:
print('외국인비율 / 자살율: ')
print(np.corrcoef(total_data['out_per'], total_data['suicide_rate']))
print('외국인비율 / 소득수준: ')
print(np.corrcoef(total_data['out_per'], total_data['per_tax']))
print('외국인비율 / 교통사고사망자: ')
print(np.corrcoef(total_data['out_per'], total_data['death']))
print('외국인비율 / 교통사고부상자: ')
print(np.corrcoef(total_data['out_per'], total_data['injured']))
print('외국인비율 / cctv: ')
print(np.corrcoef(total_data['out_per'], total_data['cctv']))

In [ ]:
print('자살율 / 소득수준: ')
print(np.corrcoef(total_data['suicide_rate'], total_data['per_tax']))
print('자살율 / 교통사고사망자: ')
print(np.corrcoef(total_data['suicide_rate'], total_data['death']))
print('자살율 / 교통사고부상자: ')
print(np.corrcoef(total_data['suicide_rate'], total_data['injured']))
print('자살율 / cctv: ')
print(np.corrcoef(total_data['suicide_rate'], total_data['cctv']))

In [ ]:
print('소득수준 / 교통사고사망자: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['death']))
print('소득수준 / 교통사고부상자: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['injured']))
print('소득수준 / cctv: ')
print(np.corrcoef(data_2016['per_tax'], data_2016['cctv']))

### 분석결과

#### 2015년

범죄율과 소득수준이 음의 약한 상관관계를 가졌고

인구수는 자살율과 음의 약한 상관관계를 ,교통사고 사망자,부상자와는 매우 강한 양의 상관관계를 보이고,

### DATA Visualation

In [ ]:
sns.pairplot(total_data, vars=["crimeRate", "suicide_rate", "per_tax", "cctv"], kind='reg', size=5)
plt.show()

In [ ]:
sns.pairplot(total_data, vars=["total",  "suicide_rate", "death","injured"], kind='reg', size=4)
plt.show()

In [ ]:
sns.pairplot(total_data, vars=["out_per", "suicide_rate", "per_tax"], kind='reg', size=3)
plt.show()

In [ ]:
sns.pairplot(total_data, vars=["suicide_rate", "per_tax", "death", "injured","cctv"], kind='reg', size=5)
plt.show()

In [ ]:
sns.pairplot(total_data, vars=["death", "injured","cctv"], kind='reg', size=3)
plt.show()

In [ ]:
#Crime Rates
data_2016['crimeRate'].sort_values().plot(kind='bar', grid=True, figsize=(20,12))
plt.xlabel('범죄율')
plt.show()

In [ ]:
#Crime Rates
data_2016['per_tax'].sort_values().plot(kind='kde',grid=True, figsize=(20,12))
plt.xlabel('범죄율')
plt.show()

In [ ]:
#per Tax
data_2016['per_tax'].sort_values().plot(kind='barh', grid=True, figsize=(10,20))
plt.xlabel('인당 세금')
plt.show()

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)

plt.plot(data_2015['district_name'], data_2015['per_tax'], '-ro')
